# 5.1. Extraindo *Features* de LLMs

* **Input**: PKL contendo todos os enunciados, alternativas e gabaritos

* **Output**: CSV com as colunas referentes as respostas que os modelos Llama-3.2 e Deepseek-v3 deram para as questões do ENEM

## 5.1.1 Extração dos dados iniciais

In [1]:
# !pip install ollama

In [2]:
# Importando as bibliotecas para lidar com LLMs
import pandas as pd
import ollama
import re
import requests
import time

In [85]:
# Caminho do CSV original + outputs finais
input_path = "../data/final/enem_data_embeddings.pkl"
output_path_llama = "llama_extraction_csv.csv"
output_path_deepseek = "deepseek_extraction_csv.csv"

In [4]:
# Carregar CSV
df = pd.read_pickle(input_path)
df.shape

(270, 16)

In [1]:
# "Quebrando" os dados em batches de 45 questões para facilitar a extração

n = 6
tamanho = len(df) // n  # 45

# Dividindo em 6 DataFrames separados
df1 = df.iloc[0*tamanho : 1*tamanho].reset_index(drop=True)
df2 = df.iloc[1*tamanho : 2*tamanho].reset_index(drop=True)
df3 = df.iloc[2*tamanho : 3*tamanho].reset_index(drop=True)
df4 = df.iloc[3*tamanho : 4*tamanho].reset_index(drop=True)
df5 = df.iloc[4*tamanho : 5*tamanho].reset_index(drop=True)
df6 = df.iloc[5*tamanho : 6*tamanho].reset_index(drop=True)

NameError: name 'df' is not defined

---
## 5.1.2. Configurando modelo LLM

In [6]:
# Definindo prompt para o modelo
prompt = """Responda à seguinte questão:

    Enunciado:
    {enunciado}

    Alternativas:
    {alternativas}

    Qual a alternativa correta? Responda explicitamente apenas com a respectiva letra (A, B, C, D ou E). (não inclua coisas como "É a letra A", quero apenas o caractere da letra)"""

### 5.1.2.1. LLama 3.2

In [ ]:
# setando os parametros para extração

LLAMA_API_KEY = "generico" 
LLAMA_API_URL = "https://openrouter.ai/api/v1/chat/completions"
MODEL = "meta-llama/llama-3.2-3b-instruct:free"

In [ ]:
def ask_llama(enunciado, alternativas, stream=False):
    """
    Envia a pergunta ao modelo Llama. Faz uma única requisição.

    Parâmetros:
    - enunciado: texto da pergunta
    - alternativas: opções de resposta
    - stream: se True, imprime chunks em tempo real
    """
    headers = {
        "Authorization": f"Bearer {LLAMA_API_KEY}",
        "Content-Type": "application/json"
    }
    payload = {
        "model": MODEL,
        "messages": [
            {"role": "user", "content": prompt.format(enunciado=enunciado, alternativas=alternativas)}
        ],
        "stream": stream,
        "max_tokens": 2048
    }

    try:
        response = requests.post(LLAMA_API_URL, headers=headers, json=payload)

        if response.status_code == 429:
            return f"Erro 429: Rate limit excedido. Tente novamente mais tarde."

        response.raise_for_status()

        if stream:
            for chunk in response.iter_lines():
                if chunk:
                    print("Chunk recebido:", chunk.decode('utf-8'))
            return None
        else:
            full_response = response.json()
            return full_response['choices'][0]['message']['content']

    except requests.exceptions.HTTPError as http_err:
        return f"Erro HTTP na API: {http_err}"
    except Exception as e:
        return f"Erro na API: {e}"


### 5.1.2.2. Deepseek-v3

In [74]:
# setando os parametros para extração

DEEPSEEK_API_KEY = "generico"
DEEPSEEK_API_URL = "https://openrouter.ai/api/v1/chat/completions"
MODEL = "deepseek/deepseek-chat-v3-0324:free"

In [75]:
def ask_deepseek(enunciado, alternativas, stream = False):
    """
    Envia a pergunta ao modelo Deepseek. Faz uma única requisição.

    Parâmetros:
    - enunciado: texto da pergunta
    - alternativas: opções de resposta
    - stream: se True, imprime chunks em tempo real
    """
    

    
    headers = {
        "Authorization": f"Bearer {DEEPSEEK_API_KEY}",
        "Content-Type": "application/json"
    }
    payload = {
        "model": MODEL,
        "messages": [
            {"role": "user", "content": prompt.format(enunciado=enunciado, alternativas=alternativas)}
        ],
        "stream": stream,
        "max_tokens": 2048
    }

    try:
        response = requests.post(DEEPSEEK_API_URL, headers=headers, json=payload)

        if response.status_code == 429:
            return f"Erro 429: Rate limit excedido. Tente novamente mais tarde."

        response.raise_for_status()

        if stream:
            for chunk in response.iter_lines():
                if chunk:
                    print("Chunk recebido:", chunk.decode('utf-8'))
            return None
        else:
            full_response = response.json()
            return full_response['choices'][0]['message']['content']

    except requests.exceptions.HTTPError as http_err:
        return f"Erro HTTP na API: {http_err}"
    except Exception as e:
        return f"Erro na API: {e}"


---
## 5.1.3. Aplicando Perguntas aos LLMs

### 5.1.3.1. Coletando Llama

In [ ]:
df1["resposta_llamma"] = df1[["enunciado", "alternativas"]].apply(
    lambda row: ask_llama(row["enunciado"], row["alternativas"]), axis=1
)

In [ ]:
df2["resposta_llamma"] = df2[["enunciado", "alternativas"]].apply(
    lambda row: ask_llama(row["enunciado"], row["alternativas"]), axis=1
)

In [ ]:
df3["resposta_llamma"] = df3[["enunciado", "alternativas"]].apply(
    lambda row: ask_llama(row["enunciado"], row["alternativas"]), axis=1
)

In [ ]:
df4["resposta_llamma"] = df4[["enunciado", "alternativas"]].apply(
    lambda row: ask_llama(row["enunciado"], row["alternativas"]), axis=1
)

In [ ]:
df5["resposta_llamma"] = df5[["enunciado", "alternativas"]].apply(
    lambda row: ask_llama(row["enunciado"], row["alternativas"]), axis=1
)

In [ ]:
df6["resposta_llamma"] = df6[["enunciado", "alternativas"]].apply(
    lambda row: ask_llama(row["enunciado"], row["alternativas"]), axis=1
)

### 5.1.3.2. Coletando Deepseek

In [34]:
df1["resposta_deepseek"] = df1[["enunciado", "alternativas"]].apply(
    lambda row: ask_deepseek(row["enunciado"], row["alternativas"]), axis=1
)


In [43]:
df2["resposta_deepseek"] = df2[["enunciado", "alternativas"]].apply(
    lambda row: ask_deepseek(row["enunciado"], row["alternativas"]), axis=1
)

In [50]:
df3["resposta_deepseek"] = df3[["enunciado", "alternativas"]].apply(
    lambda row: ask_deepseek(row["enunciado"], row["alternativas"]), axis=1
)

In [62]:
df4["resposta_deepseek"] = df4[["enunciado", "alternativas"]].apply(
    lambda row: ask_deepseek(row["enunciado"], row["alternativas"]), axis=1
)

In [69]:
df5["resposta_deepseek"] = df5[["enunciado", "alternativas"]].apply(
    lambda row: ask_deepseek(row["enunciado"], row["alternativas"]), axis=1
)

In [76]:
df6["resposta_deepseek"] = df6[["enunciado", "alternativas"]].apply(
    lambda row: ask_deepseek(row["enunciado"], row["alternativas"]), axis=1
)

## 5.1.4. Armazenando resultado

In [78]:
# concatenando novamente todos os batches

df_completo = pd.concat([df1, df2, df3, df4, df5, df6], ignore_index=True)

In [ ]:
# salvando tabela com os dados do llama

df_llama = df_completo.drop(columns='resposta_deepseek')

df_llama.to_csv('enem_llama_extraction.csv')

In [86]:
# salvando tabela com os dados do deepseek

df_deepseek = df_completo.drop(columns='resposta_llamma')

df_deepseek.to_csv('enem_deepseek_extraction.csv')

---